# Adding libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library


from bs4 import BeautifulSoup

print('Libraries imported.')



Libraries imported.


## Importing CSV


In [2]:
Germany = pd.read_csv('germany.csv')
Germany.sort_values(by = 'population', ascending = False)
Germany.drop(columns = ['country', 'city_ascii','iso2', 'iso3', 'admin_name', 'capital', 'id'], inplace = True)
Germany.head()



,city,lat,lng,population
0,Berlin,"52,5218","13,4015",3406000
1,Stuttgart,"48,78","9,2",2944700
2,Frankfurt,"50,1","8,675",2895000
3,Mannheim,"49,5004","8,47",2362000
4,Hamburg,"53,55",10,1757000


In [3]:
Germany = Germany.rename(columns = {'lat':'Latitude', 'lng': 'Longitude', 'city': 'City'})
Germany.head()



,City,Latitude,Longitude,population
0,Berlin,"52,5218","13,4015",3406000
1,Stuttgart,"48,78","9,2",2944700
2,Frankfurt,"50,1","8,675",2895000
3,Mannheim,"49,5004","8,47",2362000
4,Hamburg,"53,55",10,1757000


In [4]:
Germany_clean=Germany.drop(columns= ['population'])
Germany_clean.head()
Germany_clean['Latitude']=Germany_clean['Latitude'].apply(lambda x: x.replace(',','.'))
Germany_clean['Longitude']=Germany_clean['Longitude'].apply(lambda x: x.replace(',','.'))
Germany_clean[['Latitude','Longitude']]=Germany_clean[['Latitude','Longitude']].astype('float')
Germany_clean.dtypes

City          object
Latitude     float64
Longitude    float64
dtype: object

In [5]:
address = 'Germany'

geolocator = Nominatim(user_agent="germany")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Germany are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Germany are 51.0834196, 10.4234469.


## Creating map of Germany

In [6]:
map_germany = folium.Map(location=[latitude, longitude], zoom_start=7)

# add markers to map
for lat, lng, city in zip(Germany_clean['Latitude'], Germany_clean['Longitude'], Germany_clean['City']):
    label = ' {}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_germany)

map_germany

## Using Foursquare

In [7]:
CLIENT_ID = 'XEO5A5ZBI0P3HE4W0JPTS3UJCMQTP4WU5ZCNZHUB4ATFSV1B' # your Foursquare ID
CLIENT_SECRET = 'OT111S4TFMPLV2X1JJD1QRFO0CT3ZLXHCEB25OQ0ZH5ARIXW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: XEO5A5ZBI0P3HE4W0JPTS3UJCMQTP4WU5ZCNZHUB4ATFSV1B
CLIENT_SECRET:OT111S4TFMPLV2X1JJD1QRFO0CT3ZLXHCEB25OQ0ZH5ARIXW


In [8]:
#Function to apply it to every Postcode we have.
def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City',
                  'City Latitude',
                  'City Longitude',
                  'Venue',
                  'Venue Latitude',
                  'Venue Longitude',
                  'Venue Category']

    return(nearby_venues)



In [9]:
germany_venues = getNearbyVenues(names=Germany_clean['City'],
                                   latitudes=Germany_clean['Latitude'],
                                   longitudes=Germany_clean['Longitude']
                                  )


Berlin
Stuttgart
Frankfurt
Mannheim
Hamburg
Essen
Duisburg
Munich
Düsseldorf
Cologne
Wuppertal
Saarbrücken
Nuremberg
Bremen
Hannover
Bonn
Dresden
Wiesbaden
Dortmund
Leipzig
Heidelberg
Karlsruhe
Augsburg
Bielefeld
Koblenz
Chemnitz
Kassel
Münster
Kiel
Freiburg im Breisgau
Braunschweig
Fürth
Lübeck
Osnabrück
Magdeburg
Potsdam
Erfurt
Rostock
Mainz
Ulm
Würzburg
Oldenburg
Regensburg
Ingolstadt
Göttingen
Bremerhaven
Cottbus
Jena
Gera
Schwerin
Flensburg
Rosenheim
Gießen
Stralsund
Coburg
Hof
Emden
Passau


In [10]:
print(germany_venues.shape)
germany_venues.head()



(1133, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berlin,52.5218,13.4015,Buchhandlung Walther König,52.521301,13.400758,Bookstore
1,Berlin,52.5218,13.4015,James-Simon-Park,52.521907,13.399361,Park
2,Berlin,52.5218,13.4015,Café 93,52.522997,13.399752,Café
3,Berlin,52.5218,13.4015,Hotel ZOE,52.523095,13.400966,Hotel
4,Berlin,52.5218,13.4015,MA'LOA Poké Bowl,52.523653,13.400399,Poke Place


In [11]:
#Grouping by Neighborhood will be necessarily, and also more readable.
germany_venues.groupby('City').count().head()


,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Augsburg,6,6,6,6,6,6
Berlin,100,100,100,100,100,100
Bielefeld,24,24,24,24,24,24
Bonn,4,4,4,4,4,4
Braunschweig,1,1,1,1,1,1


In [12]:
print('There are {} uniques categories.'.format(len(germany_venues['Venue Category'].unique())))


There are 217 uniques categories.


In [13]:
# one hot encoding
germany_onehot = pd.get_dummies(germany_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
germany_onehot['City'] = germany_venues['City']

# move neighborhood column to the first column
fixed_columns = [germany_onehot.columns[-1]] + list(germany_onehot.columns[:-1])
germany_onehot = germany_onehot[fixed_columns]
germany_onehot.head()


,City,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Bavarian Restaurant,Beach,Beach Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Boat Rental,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Building,Burger Joint,Bus Station,Bus Stop,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Currywurst Joint,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,English Restaurant,Event Service,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food & Drink Shop,Fountain,Franconian Restaurant,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Manti Place,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Newsstand,Nightclub,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Pharmacy,Photography Studio,Pide Place,Pier,Pizza Place,Platform,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Record Shop,Restaurant,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Taverna,Tea Room,Tech Startup,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfront,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,Berlin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Berlin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Berlin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [14]:
germany_onehot.shape

(1133, 218)

In [15]:
#Here we are showing the mean of what each neighborhood has close by
germany_grouped = germany_onehot.groupby('City').mean().reset_index()
germany_grouped.head()

,City,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Bavarian Restaurant,Beach,Beach Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Boat Rental,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Building,Burger Joint,Bus Station,Bus Stop,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Currywurst Joint,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,English Restaurant,Event Service,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Food & Drink Shop,Fountain,Franconian Restaurant,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Manti Place,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Newsstand,Nightclub,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Pharmacy,Photography Studio,Pide Place,Pier,Pizza Place,Platform,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Record Shop,Restaurant,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Taverna,Tea Room,Tech Startup,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfront,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,Augsburg,0.0,0.00,0.0,0.000000,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.166667,0.00,0.000000,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.166667,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.166667,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.166667,0.0,0.0,0.000,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.166667,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.

In [16]:
germany_grouped.shape



(57, 218)

In [17]:
#Now, we are going to show what has more frequency in each of those neighborhoods.
num_top_venues = 5

for hood in germany_grouped['City']:
    print("----"+hood+"----")
    temp = germany_grouped[germany_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Augsburg----
                venue  freq
0          Steakhouse  0.17
1         Supermarket  0.17
2              Museum  0.17
3  Light Rail Station  0.17
4               Hotel  0.17


----Berlin----
            venue  freq
0     Art Gallery  0.06
1  Ice Cream Shop  0.05
2  Clothing Store  0.05
3           Hotel  0.05
4     Coffee Shop  0.04


----Bielefeld----
              venue  freq
0         Nightclub  0.12
1       Supermarket  0.12
2               Bar  0.08
3  Asian Restaurant  0.08
4             Hotel  0.08


----Bonn----
                        venue  freq
0                Intersection  0.25
1                    Bus Stop  0.25
2                Tech Startup  0.25
3  Construction & Landscaping  0.25
4           Accessories Store  0.00


----Braunschweig----
                   venue  freq
0            Gas Station   1.0
1      Accessories Store   0.0
2  Performing Arts Venue   0.0
3           Optical Shop   0.0
4        Organic Grocery   0.0


----Bremen----
                  ven

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]


In [19]:
#And add it into a pandas dataframe to be able to apply afterwards Kclustering.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cities_sorted = pd.DataFrame(columns=columns)
cities_sorted['City'] = germany_grouped['City']

for ind in np.arange(germany_grouped.shape[0]):
    cities_sorted.iloc[ind, 1:] = return_most_common_venues(germany_grouped.iloc[ind, :], num_top_venues)

cities_sorted.head()



,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Augsburg,Steakhouse,Hotel,Supermarket,Bus Station,Light Rail Station,Museum,Donut Shop,Drugstore,Gas Station,Garden Center
1,Berlin,Art Gallery,Ice Cream Shop,Clothing Store,Hotel,Coffee Shop,History Museum,Art Museum,Café,Park,Indie Movie Theater
2,Bielefeld,Nightclub,Supermarket,Hotel,Asian Restaurant,Bar,Middle Eastern Restaurant,English Restaurant,Metro Station,Greek Restaurant,Gym / Fitness Center
3,Bonn,Intersection,Bus Stop,Construction & Landscaping,Tech Startup,Zoo Exhibit,Flea Market,Garden Center,Garden,Furniture / Home Store,Fruit & Vegetable Store
4,Braunschweig,Gas Station,Zoo Exhibit,Fast Food Restaurant,Garden Center,Garden,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,French Restaurant,Franconian Restaurant


In [20]:
# set number of clusters, in this case we choose 3 clusters because if its >3 we will have clusters with only one data in it.
kclusters = 4

germany_grouped_clustering = germany_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(germany_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]



array([1, 1, 1, 1, 3, 1, 0, 1, 1, 1], dtype=int32)

In [21]:
# add clustering labels
cities_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

germany_merged = Germany_clean

# merge netherlands_merged with cities_sorted to add latitude/longitude for each neighborhood
germany_merged = germany_merged.join(cities_sorted.set_index('City'), on='City')

germany_merged.head() # check the last columns!



,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berlin,52.5218,13.4015,1.0,Art Gallery,Ice Cream Shop,Clothing Store,Hotel,Coffee Shop,History Museum,Art Museum,Café,Park,Indie Movie Theater
1,Stuttgart,48.7800,9.2000,1.0,Indian Restaurant,Bus Stop,Café,Scenic Lookout,Zoo Exhibit,Fast Food Restaurant,Garden,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop
2,Frankfurt,50.1000,8.6750,1.0,Café,Bakery,Bar,Restaurant,Art Museum,Italian Restaurant,Park,Drugstore,Plaza,Coffee Shop
3,Mannheim,49.5004,8.4700,1.0,Light Rail Station,Supermarket,Trattoria/Osteria,Music Venue,Mediterranean Restaurant,Bakery,Grocery Store,Convenience Store,Turkish Restaurant,Plaza
4,Hamburg,53.5500,10.0000,1.0,Coffee Shop,Hotel,Cosmetics Shop,Vietnamese Restaurant,Burger Joint,Café,German Restaurant,Restaurant,Clothing Store,Cocktail Bar


In [22]:
cities_sorted.head()


,Cluster Labels,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Augsburg,Steakhouse,Hotel,Supermarket,Bus Station,Light Rail Station,Museum,Donut Shop,Drugstore,Gas Station,Garden Center
1,1,Berlin,Art Gallery,Ice Cream Shop,Clothing Store,Hotel,Coffee Shop,History Museum,Art Museum,Café,Park,Indie Movie Theater
2,1,Bielefeld,Nightclub,Supermarket,Hotel,Asian Restaurant,Bar,Middle Eastern Restaurant,English Restaurant,Metro Station,Greek Restaurant,Gym / Fitness Center
3,1,Bonn,Intersection,Bus Stop,Construction & Landscaping,Tech Startup,Zoo Exhibit,Flea Market,Garden Center,Garden,Furniture / Home Store,Fruit & Vegetable Store
4,3,Braunschweig,Gas Station,Zoo Exhibit,Fast Food Restaurant,Garden Center,Garden,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,French Restaurant,Franconian Restaurant


In [23]:
#Deletint the NaN Cluster Labels and making them int for ploting purposes in the future.
cities_sorted.shape
cities_sorted = cities_sorted.dropna(axis='rows')
cities_sorted['Cluster Labels'] = cities_sorted['Cluster Labels'].astype(int)
cities_sorted.head()

,Cluster Labels,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Augsburg,Steakhouse,Hotel,Supermarket,Bus Station,Light Rail Station,Museum,Donut Shop,Drugstore,Gas Station,Garden Center
1,1,Berlin,Art Gallery,Ice Cream Shop,Clothing Store,Hotel,Coffee Shop,History Museum,Art Museum,Café,Park,Indie Movie Theater
2,1,Bielefeld,Nightclub,Supermarket,Hotel,Asian Restaurant,Bar,Middle Eastern Restaurant,English Restaurant,Metro Station,Greek Restaurant,Gym / Fitness Center
3,1,Bonn,Intersection,Bus Stop,Construction & Landscaping,Tech Startup,Zoo Exhibit,Flea Market,Garden Center,Garden,Furniture / Home Store,Fruit & Vegetable Store
4,3,Braunschweig,Gas Station,Zoo Exhibit,Fast Food Restaurant,Garden Center,Garden,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,French Restaurant,Franconian Restaurant


In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(germany_merged['Latitude'], germany_merged['Longitude'], germany_merged['City'], cities_sorted['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [25]:
#Cluster 0
germany_merged.loc[germany_merged['Cluster Labels'] == 0, germany_merged.columns[[0] + list(range(4, germany_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Wuppertal,Pool,German Restaurant,Café,Sculpture Garden,Fast Food Restaurant,Garden,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,French Restaurant
44,Göttingen,German Restaurant,Lake,Zoo Exhibit,Flea Market,Gas Station,Garden Center,Garden,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop
45,Bremerhaven,Drugstore,Café,Zoo Exhibit,Flea Market,Gas Station,Garden Center,Garden,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop


In [26]:

#Cluster 1
germany_merged.loc[germany_merged['Cluster Labels'] == 1, germany_merged.columns[[0] + list(range(4, germany_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berlin,Art Gallery,Ice Cream Shop,Clothing Store,Hotel,Coffee Shop,History Museum,Art Museum,Café,Park,Indie Movie Theater
1,Stuttgart,Indian Restaurant,Bus Stop,Café,Scenic Lookout,Zoo Exhibit,Fast Food Restaurant,Garden,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop
2,Frankfurt,Café,Bakery,Bar,Restaurant,Art Museum,Italian Restaurant,Park,Drugstore,Plaza,Coffee Shop
3,Mannheim,Light Rail Station,Supermarket,Trattoria/Osteria,Music Venue,Mediterranean Restaurant,Bakery,Grocery Store,Convenience Store,Turkish Restaurant,Plaza
4,Hamburg,Coffee Shop,Hotel,Cosmetics Shop,Vietnamese Restaurant,Burger Joint,Café,German Restaurant,Restaurant,Clothing Store,Cocktail Bar
5,Essen,Hotel,Drugstore,Cocktail Bar,Bakery,Department Store,Coffee Shop,Men's Store,Metro Station,Middle Eastern Restaurant,Furniture / Home Store
6,Duisburg,Snack Place,Turkish Restaurant,Theater,Zoo Exhibit,Fast Food Restaurant,Garden Center,Garden,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop
7,Munich,Café,Cocktail Bar,Bar,Supermarket,Pizza Place,Breakfast Spot,Coffee Shop,Afghan Restaurant,Vietnamese Restaurant,Asian Restaurant
8,Düsseldorf,Café,Italian Restaurant,Boutique,Coffee Shop,Plaza,Vegetarian / Vegan Restaurant,Shopping Mall,Furniture / Home Store,Steakhouse,German Restaurant
9,Cologne,Hotel,Café,Pub,Event Space,Restaurant,Sushi Restaurant,Water Park,Plaza,German Restaurant,Dance Studio


In [27]:
#Cluster 2
germany_merged.loc[germany_merged['Cluster Labels'] == 2, germany_merged.columns[[0] + list(range(4, germany_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Koblenz,German Restaurant,Hotel,Beer Garden,Park,Zoo Exhibit,Flea Market,Garden Center,Garden,Furniture / Home Store,Fruit & Vegetable Store
29,Freiburg im Breisgau,Hotel,Zoo Exhibit,Farmers Market,Garden Center,Garden,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,French Restaurant,Franconian Restaurant
43,Ingolstadt,Hotel,Insurance Office,Supermarket,Café,Convenience Store,Bakery,Flea Market,Garden Center,Garden,Furniture / Home Store
56,Emden,Hotel,Steakhouse,Hostel,Fast Food Restaurant,Gas Station,Garden Center,Garden,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop


In [28]:

#Cluster 3
germany_merged.loc[germany_merged['Cluster Labels'] == 3, germany_merged.columns[[0] + list(range(4, germany_merged.shape[1]))]]


,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Braunschweig,Gas Station,Zoo Exhibit,Fast Food Restaurant,Garden Center,Garden,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,French Restaurant,Franconian Restaurant
